In [1]:
import pandas as pd
import numpy as np

time: 16.9 s


In [2]:
df = pd.read_csv('Capstone_oversampled.csv')

time: 6.99 s


In [4]:
X = df.drop('readmitted',axis=1)
y = df['readmitted']

time: 111 ms


from sklearn.decomposition import PCA
pca=PCA(random_state=3)
pca.fit(X)
pd.DataFrame({'Eigenvalue':pca.explained_variance_,'Proporsion Explained':pca.explained_variance_ratio_,'Cummumlatice Proportion Exaplaine':np.cumsum(pca.explained_variance_ratio_)})

pca=PCA(n_components=0.95,random_state=96)
X_pc=pca.fit_transform(X)
X_pc.shape

In [6]:
from pyforest import *

time: 10 s


X_train,X_test,y_train,y_test = train_test_split(X_pc,y,test_size=0.30,random_state=96)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=96)

<IPython.core.display.Javascript object>

time: 42.5 s


In [9]:
import scipy.stats as st

time: 6 ms


In [11]:
st.ttest_ind(X_train,X)[1]

array([0.60861642, 0.63535103, 0.97260734, 0.90838605, 0.71287632,
       0.87071257, 0.79839155, 0.64156096, 0.84003783, 0.36430649,
       0.70965344, 0.46479894, 0.89857823, 0.85081891, 0.98115089,
       0.82843596, 0.84452379, 0.8024579 , 0.68127983, 0.72343124,
       0.97766794, 0.82031886, 0.64490064, 0.68946637, 0.44991251])

time: 373 ms


In [12]:
st.ttest_ind(X_test,X)[1]

array([0.37536906, 0.40754525, 0.9522714 , 0.8401109 , 0.52047598,
       0.77639898, 0.65462137, 0.41508488, 0.7239128 , 0.11734816,
       0.51597102, 0.20058426, 0.82365472, 0.7408209 , 0.96700341,
       0.70314978, 0.73447463, 0.66238657, 0.47306856, 0.53683842,
       0.96099517, 0.69144533, 0.4212215 , 0.48539057, 0.18672249])

time: 226 ms


In [6]:
from sklearn.pipeline import Pipeline

time: 45 ms


In [7]:
from xgboost.sklearn import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
from scipy.stats import randint
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.svm import SVC


time: 4.34 s


In [8]:
# Create a pipeline
pipe = Pipeline([("classifier", XGBClassifier())])
# Create dictionary with candidate learning algorithms and their hyperparameters
r_param = [{"classifier": [LogisticRegression()],
                 "classifier__penalty": ['l2','l1'],
                 "classifier__C": np.logspace(0, 4, 10)},
               {"classifier": [LogisticRegression()],
                 "classifier__penalty": ['l2'],
                 "classifier__C": np.logspace(0, 4, 10),
                 "classifier__solver":['newton-cg','saga','sag','liblinear'] ##This solvers don't allow L1 penalty
                 },
            {"classifier": [DecisionTreeClassifier()],
             "classifier__criterion":['gini','entropy'],
                 "classifier__max_depth":[5,8,15,25,30,None],
                 "classifier__min_samples_leaf":[1,2,5,10,15,100],
                 "classifier__max_leaf_nodes": [2, 5,10]},
               {"classifier": [RandomForestClassifier()],
                 "classifier__n_estimators": [10, 100, 1000],
                 "classifier__max_depth":[5,8,15,25,30,None],
                 "classifier__min_samples_leaf":[1,2,5,10,15,100],
                 "classifier__max_leaf_nodes": [2, 5,10]},
           {'classifier':[lgb.LGBMClassifier()],
            'classifier__n_estimators':np.arange(50,250,5),
            'classifier__max_depth':np.arange(2,15,5),
            'classifier__num_leaves':np.arange(2,60,5)},
           {'classifier':[XGBClassifier()],
              "classifier__learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
            "classifier__min_child_weight" : [ 1, 3, 5, 7 ],
            "classifier__gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
            "classifier__colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]},
           #{'classifier':[SGDClassifier()],                                     ##Stocasticated Gradient decent
            #"classifier__C": [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0],
            #"classifier__penalty": ['l2']},
           {'classifier':[AdaBoostClassifier()],        ## Adaboost Classifier
           "classifier__n_estimators": sp_randint(50,250), 
            'classifier__learning_rate': [(0.97 + x / 100) for x in range(0, 8)],
            'classifier__algorithm': ['SAMME', 'SAMME.R']},
           {'classifier':[KNN()],
            "classifier__weights":['uniform','distance'],
            'classifier__n_neighbors':np.arange(1,40),
            'classifier__leaf_size':np.arange(2,40)},
           {'classifier':[SVC()],                        ## SVM Algorithm
           'classifier__gamma':np.logspace(-4,2,10000),
       'classifier__C':np.logspace(-2,2,10000)},
           {"classifier":[GradientBoostingClassifier()],
            "classifier__learning_rate":np.arange(0,0.5,15),
            "classifier__n_estimators":np.arange(50,250,5),
            'classifier__max_depth':np.arange(2,15,5),
            "classifier__min_samples_leaf":[1,2,5,10,15,100],
            "classifier__max_leaf_nodes": [2, 5,10]}
          ]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

time: 118 ms


In [9]:
from sklearn.model_selection import RandomizedSearchCV


time: 5.01 ms


In [10]:
rsearch = RandomizedSearchCV(pipe, r_param, cv=5, verbose=0,n_jobs=-1,random_state=96)

time: 4 ms


In [11]:
best_model_r = rsearch.fit(X_train,y_train)

time: 23min 44s


In [12]:
print(best_model_r.best_estimator_)
print("The mean accuracy of the model is through randomized search is :",best_model_r.score(X_test,y_test))

Pipeline(steps=[('classifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=0.3, gamma=0.0, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='', learning_rate=0.3,
                               max_delta_step=0, max_depth=6,
                               min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=0, num_parallel_tree=1,
                               objective='multi:softprob', random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
                               subsample=1, tree_method='exact',
                               validate_parameters=1, verbosity=None))])
The mean accuracy of the model is through randomized search is : 0

In [13]:
best_model_r.best_params_

{'classifier': XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
               colsample_bynode=None, colsample_bytree=0.3, gamma=0.0,
               gpu_id=None, importance_type='gain', interaction_constraints=None,
               learning_rate=0.3, max_delta_step=None, max_depth=None,
               min_child_weight=1, missing=nan, monotone_constraints=None,
               n_estimators=100, n_jobs=None, num_parallel_tree=None,
               random_state=None, reg_alpha=None, reg_lambda=None,
               scale_pos_weight=None, subsample=None, tree_method=None,
               validate_parameters=None, verbosity=None),
 'classifier__colsample_bytree': 0.3,
 'classifier__gamma': 0.0,
 'classifier__learning_rate': 0.3,
 'classifier__min_child_weight': 1}

time: 17.1 ms


In [14]:
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score,roc_curve,classification_report


time: 5 ms


In [15]:
XGBC=XGBClassifier(**best_model_r.best_params_,random_state=96)
XGBC.fit(X_train,y_train)
y_train_pred=XGBC.predict(X_train)
y_train_prob=XGBC.predict_proba(X_train)[:,1]

y_test_pred=XGBC.predict(X_test)
y_test_prob=XGBC.predict_proba(X_test)[:,1]

print('Confusion Matrix-Train\n',confusion_matrix(y_train,y_train_pred))
print('Accuracy Score-Train\n',accuracy_score(y_train,y_train_pred))
print('Classification Report-Train\n',classification_report(y_train,y_train_pred))

print('\n'*2)
print('Confusion Matrix-Test\n',confusion_matrix(y_test,y_test_pred))
print('Accuracy Score-Test\n',accuracy_score(y_test,y_test_pred))
print('Classification Report-Test\n',classification_report(y_test,y_test_pred))



[17:05:41] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { classifier, classifier__colsample_bytree, classifier__gamma, classifier__learning_rate, classifier__min_child_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Confusion Matrix-Train
 [[32480  5801   145]
 [12955 16962  8439]
 [ 4368  4170 29888]]
Accuracy Score-Train
 0.6885806541212416
Classification Report-Train
               precision    recall  f1-score   support

           0       0.65      0.85      0.74     38426
           1       0.63      0.44      0.52     38356
           2       0.78      0.78      0.78     38426

    accuracy                           0.69    115208
   macro avg       0.69      0.69      0.68    115208
weighted avg       0.6